# SIFT based Image Stitching
    
Welcome to PA5, in this week we will be studying how to stich multiple images to form a panoramic image. In particular, we will use two images coming from the same scene and will match them by using SIFT features in this assignment in four steps:

-  Step 1: First detect keypoints (DoG, Harris, etc.) and extract local invariant descriptors (SIFT, SURF, etc.) from the two input images. We will use features computed by the DoG approach (as they are computed in SIFT).
-  Step 2: Then, we will match the computed keypoints between the two images. We will use David Lowe's technique to match the features.
-  Step 3: To project one image onto another, we will compute (estimate) the homography matrix. For computing the H matrix, we will use the RANSAC algorithm on our matched feature vectors.
-  Step 4: Finally, we will apply a warping transformation using the H matrix obtained in Step 3 to combine both images.

In this assignment you'll use OpenCV, an essential library for many computer vision applications in Python.

## Prerequisite Packages

You may need to install the OpenCV with the following commands 

    pip install opencv-python==3.4.2.16
    pip install opencv-contrib-python==3.4.2.16
    pip install imutils

In [ ]:

# Run the three pip commands below to install openCV related libraries if you do not have the necessary openCV libraries in your system!

# You can run those 3 pip commands in your console seperately too before starting the jupyter notebook!

# If you are using Google Colab to run this notebook, then set the following variable useGoogleColab = 1 
# If you do not need to install those openCV libraries, then set: useGoogleColab = 0
useGoogleColab = 0

if useGoogleColab == 1:
    !pip install opencv-python==3.4.2.16
    !pip install opencv-contrib-python==3.4.2.16
    !pip install imutils

# import the necessary packages
import numpy as np
import imutils
import cv2

%matplotlib inline

import matplotlib.pyplot as plt

## Introduction

First, lets have a look at two images that we will use in this assignment: 
    

Image 1 | Image 2
:------------------------:|:-----------------------:
<img src="images/bryce_left_02.png" style="width:200px;height:150px;">  |  <img src="images/bryce_right_02.png" style="width:200px;height:150px;">

The final result should look like this image:

Panorama |
:------------------------:|
<img src="images/result.png" style="width:350px;height:150px;"> |

### Topics Covered
Lets first visualize those images on our own system with the following code:

In [ ]:
# Let us read the input images first.
#if you are using Google Colab to run this notebook, then set the following variable useGoogleColab = 1 
useGoogleColab = 1
if useGoogleColab == 0:
    image1 = cv2.imread("./images/bryce_left_02.png")
    image2 = cv2.imread("./images/bryce_right_02.png")
else:
    !wget "https://github.com/Rice-Field/SIFT-Panorama-Stitch/blob/master/images/bryce_left_02.png?raw=1"
    !wget "https://github.com/Rice-Field/SIFT-Panorama-Stitch/blob/master/images/bryce_right_02.png?raw=1"
    image1 = cv2.imread("bryce_left_02.png?raw=1")
    image2 = cv2.imread("bryce_right_02.png?raw=1")

print("Original image dimension: " + str(image1.shape))

# We keep the images small to keep the computations short
image1 = imutils.resize(image1, width=400)
image2 = imutils.resize(image2, width=400)

print("Resized image dimension: " + str(image1.shape))

# OpenCV loads images in BGR format. Lets convert that to our RGB format that we are more comfortable with.
image1_RGB = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
plt.figure()
plt.imshow(image1_RGB)
plt.show()

image2_RGB = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)
plt.figure()
plt.imshow(image2_RGB)
plt.show()

## 1. Detect Key Points

Now that we can read and plot both images in this Jupyter notebook, we should start with initializing a SIFT descriptor object. In openCV, we already have a default function that initializes the SIFT object. Using that SIFT object, we can compute the keypoints and their 128 dimensional features (also known as descriptors). After that, we convert the CV keypoint objects into numpy arrays for visualization and for further processing. Each sift keypoint has a 128 dimensional feature vector. In other words, each key point is described with 128 dimensional vector (Each keypoint is represented locally with its 4 x 4 neighbor blocks where each block is represented by 8 dimensional histograms, thus 4x4x8=128).See the lecture slides for further details.

| Difference of Gaussian    | SIFT Descriptor       |                
|:------------------------:|:-----------------------:|
| <img src="figs/dog.PNG" style="width:200px;height:150px;"> | <img src="figs/descriptor.PNG" style="width:300px;height:150px;"> |
| | [source](https://opencv-python-tutroals.readthedocs.io) |


**Functions:**
We will use the following functions that come under openCV in this exercise:
    
    cv2.xfeatures2d.SIFT_create()
    object.detectAndCompute(image, None)
    cv2.drawKeypoints(input_image, keypoints, output_image)

<div class="alert alert-info"><h1>Exercise 1</h1>
    <p> use OpenCV's sift computing function to compute the keypoints (<code>kps1</code> and <code>kps2</code>) and their descriptors (<code>features1</code> and <code>features2</code> ) for both images below. Then plot those keypoints on each image. </p>
</div>


In [ ]:
# initialize DoG and SIFT
descriptor = cv2.xfeatures2d.SIFT_create()

# Extract features from each image
#----- code below -------
# ~ one line of code
(kps1, features1) = None
#~ one line of code
(kps2, features2) = None
#---- end of code -------

# Show the detected keypoints on both images
draw1 = image1.copy()
cv2.drawKeypoints(draw1, kps1, draw1)
draw1_RGB = cv2.cvtColor(draw1, cv2.COLOR_BGR2RGB)
plt.figure()
plt.imshow(draw1_RGB)
plt.show()

draw2 = image2.copy()
cv2.drawKeypoints(draw2, kps2, draw2)
draw2_RGB = cv2.cvtColor(draw2, cv2.COLOR_BGR2RGB)
plt.figure()
plt.imshow(draw2_RGB)
plt.show()


# convert the keypoint types from openCV's KeyPoint objects to NumPy arrays
kps1 = np.float32([kp.pt for kp in kps1])
kps2 = np.float32([kp.pt for kp in kps2])

print("Feature descriptor length: " + str(len(features1[0])))
print("Keypoint length: " + str(len(kps1[0])))

**Expected ouput:**

|Draw 1 | Draw 2 |             
|:------------------------:|:-----------------------:|
| <img src="images/draw1.png" style="width:200px;height:150px;"> | <img src="images/draw2.png" style="width:200px;height:150px;"> |

## 2. Matching Points

Now that we have keypoints and their descriptors for each image computed, we need to find the keypoint pairs that exist in both images. For that purpose, we initialize a object matcher with "Brute Force". This parameter ensures that each feature from image1 is compared to all the features from the second image. We then use the k-nearest neighbor algorithm, with k = 2, to find the top two matches for each points.

We then apply Lowe's ratio test to select matches where the best match is a ratio better than the second best.

**Functions:**

We will use the following functions that come under openCV in this exercise:
    
    cv2.DescriptorMatcher_create("BruteForce")
    object.knnMatch(features A, features B, k)
    
    
    
<div class="alert alert-info"><h1>Exercise 2</h1>
    <p> use openCV to compute the matching ke-ypoint pairs (<code>initialMatches</code>) between both images below. </p>
</div>



In [ ]:
# compute the raw matches and initialize the list of actual matches
matcher = cv2.DescriptorMatcher_create("BruteForce")

#------ write your code below -------
# ~one line of code
initialMatches = None
#------- end of your code ---------

matches = []

ratio=0.75
# loop over the intial matches and apply Lowe's ratio test to see if they are good matches.
for m in initialMatches:
    # check to see if the distance is within a certain ratio of each
    # other (i.e. Lowe's ratio test)
    if len(m) == 2 and m[0].distance < m[1].distance * ratio:
        matches.append((m[0].trainIdx, m[0].queryIdx))
        

Expected output: No output here!

## 3. Homography Matrix: 

With our computed keypoint matches, we can now compute the Homography (H) matrix necessary to project one image onto the other. We will use RANSAC algorithm to compute the H matrix. 

|[Source](https://docs.opencv.org/3.4.1/d9/dab/tutorial_homography.html) |          
|:------------------------:|
| <img src="figs/homography.jpg" style="width:500px;height:150px;"> |

**Functions:**

    cv2.findHomography(pointset1, pointset2, cv2.RANSAC, Ransac threshold)
    cv2.circle(image, points, radius, (B,G,R), thickness)





<div class="alert alert-info"><h1>Exercise 3</h1>
    <p> use openCV to compute Homography matrix H (<code>(H, status)</code>) by using the RANSAC algorithm. Then, plot only the matching keypoints on each image individually (<code>match1</code> and <code>match2</code>). </p>
</div>




In [ ]:
# computing a homography requires at least 4 matches
# use the ransacThresh value given below to compute the H matrix within RANSAC
if len(matches) > 4:
    # construct the two sets of keypoints as np arrays
    ransacThresh=4.0
    pts1 = np.float32([kps1[i] for (_, i) in matches])
    pts2 = np.float32([kps2[i] for (i, _) in matches])

    # compute the homography between the two sets of points by using RANSAC
    #------ write your code below -------
    # ~ one line of code
    (H, status) = None
    #------ end of your code -------

match1 = image1.copy()
match2 = image2.copy()
# loop over the matches
for ((trainIdx, queryIdx), s) in zip(matches, status):
    # only process the match if the keypoint was successfully
    # matched
    if s == 1:
        # draw the match
        pt1 = (int(kps1[queryIdx][0]), int(kps1[queryIdx][1]))
        pt2 = (int(kps2[trainIdx][0]), int(kps2[trainIdx][1]))
        
        # use the color red for circles
        #------ write your code below -------
        # ~one line of code 
        match1 = None
        # ~one line of code
        match2 = None
        #------ end of your code -------
        
match1_RGB = cv2.cvtColor(match1, cv2.COLOR_BGR2RGB)
plt.figure()
plt.imshow(match1_RGB)
plt.show()

match2_RGB = cv2.cvtColor(match2, cv2.COLOR_BGR2RGB)
plt.figure()
plt.imshow(match2_RGB)
plt.show()

**Expected ouput:**

|Draw 1 | Draw 2 |             
|:------------------------:|:-----------------------:|
| <img src="images/match1.png" style="width:200px;height:150px;"> | <img src="images/match2.png" style="width:200px;height:150px;"> |



## Visualization of only the matching key-points

Now that we know what the stiched image looks like, lets have a look at what features matched in both images. For, that, we will visualize the matching features on both images shown side by side. For that we will pick all the matching key point-pairs and visualize them pair by pair as a line. In this way, the left side of the image will show the first image, and the right hand side will show the second image. Since we need two points to draw a line, all we need is actually those individual matching pairs to draw matching lines. See the expected results below. You should see mostly parallel lines.

**Functions:**

    cv2.line(image, point 1, point 2, (0, 255, 0), thicknessVal)
    



<div class="alert alert-info"><h1>Exercise 4</h1>
    <p> use openCV to plot the new image containing the matching keypoints only  (<code>initialMatches</code>) between both images below. </p>
</div>




In [ ]:
# initialize the output visualization image
(h1, w1) = image1.shape[:2]
(h2, w2) = image2.shape[:2]
vis = np.zeros((max(h1, h2), w1 + w2, 3), dtype="uint8")
vis[0:h1, 0:w1] = match1
vis[0:h2, w1:] = match2

# loop over the matches
for ((trainIdx, queryIdx), s) in zip(matches, status):
    # only process the match if the keypoint was successfully
    # matched
    if s == 1:
        # draw the match
        pt1 = (int(kps1[queryIdx][0]), int(kps1[queryIdx][1]))
        pt2 = (int(kps2[trainIdx][0]) + w1, int(kps2[trainIdx][1]))

        # use the color green for lines
        #------ write your code below -------
        vis = None
        # ------ end of your code -----------
        
vis_RGB = cv2.cvtColor(vis, cv2.COLOR_BGR2RGB)
plt.figure()
plt.imshow(vis_RGB)
plt.show()

**Expected ouput:**

|Draw 1 |          
|:------------------------:|
| <img src="images/vis.png" style="width:400px;height:150px;"> |




## 4. Warp and Stitch

Finally after computing the homography matrix, we warp the second image (called image1) on to the first one (called image2). Meanwhile, we also ensure that the size of the final (stiched) image can fit the both images to be stiched. 

**Functions:** You can use openCV's built-in function warpPerspective here (see below).

    cv2.warpPerspective(image2, HomographyMatrix,(image2_width + image1_width, image2_height))
    


<div class="alert alert-info"><h1>Exercise 5</h1>
    <p> Given the computed H matrix and both images, use openCV to stitch both images into a single image  by projecting one image onto the other one and obtain the final stiched image (<code>result</code>) below. </p>
</div>





In [ ]:
# (matches, H, status) = M
#------ write your code below -------
# ~ one line of code
result = None
#------ end of your code -------

result[0:image1.shape[0], 0:image1.shape[1]] = image1
        
#---- Now visualize the final stiched image ---
result_RGB = cv2.cvtColor(result, cv2.COLOR_BGR2RGB)
plt.figure()
plt.imshow(result_RGB)
plt.show()

**Expected ouput:**

|Draw 1 |          
|:------------------------:|
| <img src="images/result.png" style="width:400px;height:150px;"> |

Congratulations! You now know how to compute SIFT features, how to match them and how to stich images! You also practiced how to compute the homogprahy matrix by using RANSAC now.